In [ ]:
import numpy as np

F = np.array([
    [0.1, 0.2, -0.2, 0.4],
    [-0.1, -0.2, -0.2, 0.8],
    [0.1, 0.3, -0.2, 0.5],
    [-0.1, 0.4, -0.2, 0.4],
    [0.1, 0.6, -0.2, 0.2],
])

H, W = F.shape
pooled_h = pooled_w = 2
bin_h = H / pooled_h
bin_w = W / pooled_w

roipool = np.zeros((pooled_h, pooled_w))
for ph in range(pooled_h):
    for pw in range(pooled_w):
        y_start = int(np.floor(ph * bin_h))
        y_end   = int(np.floor((ph + 1) * bin_h))
        x_start = int(np.floor(pw * bin_w))
        x_end   = int(np.floor((pw + 1) * bin_w))
        print(f"ph : {ph}, pw: {pw}, y_start: {y_start}, y_end: {y_end},  x_start: {x_start}, x_end: {x_end}")
        roipool[ph, pw] = np.max(F[y_start:y_end, x_start:x_end])
roipool

ph : 0, pw: 0, y_start: 0, y_end: 2,  x_start: 0, x_end: 2
ph : 0, pw: 1, y_start: 0, y_end: 2,  x_start: 2, x_end: 4
ph : 1, pw: 0, y_start: 2, y_end: 5,  x_start: 0, x_end: 2
ph : 1, pw: 1, y_start: 2, y_end: 5,  x_start: 2, x_end: 4


array([[0.2, 0.8],
       [0.6, 0.5]])

In [ ]:
def bilinear_sample(F, y, x):
    H, W = F.shape
    # clamp x,y to [0,H-1], [0,W-1]? In ROIAlign they'd clip. But our sample points inside range.
    if x < 0: x = 0
    if y < 0: y = 0
    if x > W-1: x = W-1
    if y > H-1: y = H-1
    x0 = int(np.floor(x))
    x1 = min(x0+1, W-1)
    y0 = int(np.floor(y))
    y1 = min(y0+1, H-1)
    alpha = x - x0
    beta  = y - y0
    val = (1-alpha)*(1-beta)*F[y0,x0] + alpha*(1-beta)*F[y0,x1] + (1-alpha)*beta*F[y1,x0] + alpha*beta*F[y1,x1]
    return val

pooled_h=pooled_w=2
bin_h = H/pooled_h
bin_w = W/pooled_w

roi_align = np.zeros((pooled_h, pooled_w))
for ph in range(pooled_h):
    for pw in range(pooled_w):
        x_center = (pw + 0.5)*bin_w
        y_center = (ph + 0.5)*bin_h
        roi_align[ph,pw] = bilinear_sample(F, y_center, x_center)
roi_align


array([[-0.075,  0.725],
       [ 0.55 ,  0.25 ]])

- Determine center of each bin
- Determine the closest 4 points in the (x,y) plane
- Use in formula to estimate the bin value.

